## Instruct Fine-tuning

В предыдущих семинарах мы в основном фокусировались на какой-то одной задаче - исправление опечаток, выделение сущностей, определение тональности, машинный перевод и т.п. Мы двигались от самых простых моделей к предобученным трансформерам, но подход в целом не менялся - мы брали модель и тренировали её решать нужную задачу, а когда нужно было решить новую задачу мы откладывали старую модель и тренировали новую. 

Одно из главных изменений в NLP в последние несколько лет - это то, что решать разные задачи стало возможно одной общей моделью. Если задуматься, то все к этому шло:

1) Self-supervised предобучение научились более менее стабильно масштабировать на огромные текстовые корпуса, что дало нам модели, в которых уже заложено очень широкое понимание языка
2) трасформеры оказались очень масштабируемой архитектурой, что позволило масштабировать не только данные, но и размер модели; и оказалось, что увеличение размера модели дает нелинейный прирост в качестве
3) При правильных промтах предобученный модели могли решать даже задачи, которые они никогда не видели (zero-shot и in context learning)
4) Fine-tuning предобученных моделей под конкретную задачу требует небольшое количество примеров (сотни или даже десятки)
5) Все возможные NLP задачи стали сводится к генерации текста (классификация - генерация одного токена, исправление опечаток - генерация исправленной последовательности, выделение сущности - генерация именованых сущностей нужного типа, даже генерация кода теперь решается просто генерацией)


Поэтому было вопросом времени, когда кто-то попробует обучить модель решать сразу все нужные задачи и у них получится.

Текущее стандартное решение - дообучить модель на датесете разнообразных инструкций, которые соответствуют нужным задачам, а затем дотренировать новую модель с помощью Reinforcement Learning from Human Feedback (RLHF). Такой подход первым успешно применил OpenAI и результат можно наблюдать в ChatGPT (спойлер: результат очень хороший). Но OpenAI не опубликовал в открытом доступе ни модели, ни код ни подробные технические описания их подхода. Поэтому сейчас болшАя часть исследовательского сообщества в NLP занимается тем, что пытается воспроизвести chatgpt по общем описаниям, которые раскрыл OpenAI. И очень многое уже получилось воспроизвести и буквально с каждым днем такие модели становятся меньше/дешевле и доступнее.

Помимо OpenAI стоит упомянуть еще две статьи (и модели), которые есть в открытом доступе и которые сильно повлияли на движение в сторону общих моделей.

## T5

![](https://1.bp.blogspot.com/-o4oiOExxq1s/Xk26XPC3haI/AAAAAAAAFU8/NBlvOWB84L0PTYy9TzZBaLf6fwPGJTR0QCLcBGAsYHQ/s640/image3.gif)


Первая статья Т5 (Text-To-Text Transfer Transformer, https://arxiv.org/abs/1910.10683, Google Research, конец 2019 года) 
Это очень большая статья, в которой подробно исследовалась унификация различных NLP задач в задачу генерации. А также они попробовали много различных подходов к предобучению (в то время выходило очень много статей, которые как-то меняли self-supervised задачу и они попробовали много разных комбинаций, чтобы получить хорошую модель). В результате у них получилось несколько вариантов модели Т5 и всех их они выложили в открытый доступ. 
Еще в статье они пробовали тюнить модель под разные задачи, но по большей части все еще по отдельности. Если вы прочитаете статью или хотя бы описание fine-tuning экспериментов, то увидите, что на тот момент парадигма (1 модель - 1 задача) еще не изменилась. В своих экспериментах они пробовали тренироваться сразу под несколько задач, но у них было не достаточно много разнообразных задач и в итоге общая модель работала хуже на отдельных задачах, чем специфичные модели.

Но в открытый доступ они выложили в том числе и модели, которые были дообучены на нескольких задачах. Задача в модель передается через префикс (посмотрите на начало примеров выше). Эти модели есть на huggingface, давайте попробуем взять какую-то модель и попробовать сходу решить задачу саммаризации.


In [1]:
import pandas as pd
import numpy as np
import torch
import json

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
# MODEL_NAME = 't5-large'
MODEL_NAME = 't5-base'
# MODEL_NAME = 't5-small'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Возьмем какой-нибудь текст

In [4]:
task_prefix = "summarize: {}"

text = """
Badgers burrowing under rail tracks have halted trains in the northern and southern Netherlands, forcing lengthy cancellations on at least two lines.
All trains were halted Tuesday afternoon on a busy line between the southern cities of Den Bosch and Boxtel after the animals dug into a dike carrying rails. The national railway company said the line would be out of service for at least a week.
The digging means "the rails can subside and then the safety of train traffic can no longer be guaranteed," ProRail, the company that maintains the Dutch rail network said in a statement.
Earlier this month, badgers also burrowed under tracks near the northern village of Molkwerum in Friesland province, knocking a line out of service until next month while workers seek permission to shift the animals.
Badgers are protected animals in the Netherlands, so rail operators have to get permission to move them or disturb their habitat before repairs can begin.
"""

С моделями в huggingface удобнее всего работать через torch, но это не страшно, так как все основные вещи реализованы в transformers и они одинаковые для torch и tf. 

Попробуем сгенерировать саммари

In [5]:
inputs = tokenizer([task_prefix.format(text)], 
                    return_tensors="pt", padding=True)

output_sequences = model.generate(
    # this parameters are also important but you can read about them in the docs and just try changing them
    num_beams=5,
    max_length=100,
    no_repeat_ngram_size=3, 
#     repetition_penalty= 5.0,
#     length_penalty=0.01,
#     early_stopping=True,
#     do_sample=True, 
#     top_k=30, 
#     top_p=0.8, 
    early_stopping=True,
#     num_return_sequences=3,
    num_return_sequences= 1,
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,  # disable sampling to test if batching affects output
)

In [6]:
summaries = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
summaries

['all trains halted on a busy line between den Bosch and boxtel . badgers dug into a dike carrying rails . the national railway company says the line will be out of service for at least a week .']

Работает неплохо, но конечно для реального практическо применения нужно тюнить модель дополнительно

## Flan T5

![](https://1.bp.blogspot.com/-_kPdaMrcRWI/YV2b-XFoRxI/AAAAAAAAIMw/KDjg0IfuoK8hjpSXNODoV46D8Rb5rK8hgCLcBGAsYHQ/w640-h178/image3.gif)


Второя статья - FLAN (тоже от Google Research, тоже огромная, Finetuned Language Models Are Zero-Shot Learners, https://arxiv.org/abs/2109.01652, середина 2021 года)

В этой статье уже заметен сдвиг в сторону общих моделей и уже сформировался подход к такому обучению через инструкции. Основная идея в статье - переделать различные NLP датасеты в большой датасет разнообразных инструкций (они сделали различные темплейты на правилах и прогнали их через размеченные датасеты) и обучить модель решать сразу всё. Инструкции при этом это не какие-то технические теги как в T5, а нормальные человеческие инструкции (буквально что-то вроде "Translate this text from English to Russian", "Write five topics that describe this text", "What is the sentiment of this text? Options: Negative, Positive, Neutral."). При таком подходе они заметили, что модель начинает обобщаться на инструкции, которых она никогда не видела - так как модель предобучена на большом количестве текстов, она уже хорошо понимает язык и экстраполирует инструкции из обучающей выборки, используя свое понимание языка). И чем больше таких инструкций, тем лучше получалось.

Они попробовали такой подход с разными моделями (T5, PALM) и везде получалось хорошо решать новые задачи.

In [7]:
import pandas as pd
import numpy as np
import torch
import json

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [8]:
MODEL_NAME = 'google/flan-t5-small'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Инструкции модели можно передавать в свободном формате, поэтому сделаем функцию, чтобы удобнее было пробовать разные инструкции.

In [10]:
def predict_for_instruction(instruction, text, model):
    

    inputs = tokenizer([instruction.format(text)], 
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=5,
        max_length=100,
        no_repeat_ngram_size=3, 
    #     repetition_penalty= 5.0,
    #     length_penalty=0.01,
    #     early_stopping=True,
    #     do_sample=True, 
    #     top_k=30, 
    #     top_p=0.8, 
        early_stopping=True,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
    return summaries[0]

In [11]:
text = """
Badgers burrowing under rail tracks have halted trains in the northern and southern Netherlands, forcing lengthy cancellations on at least two lines.
All trains were halted Tuesday afternoon on a busy line between the southern cities of Den Bosch and Boxtel after the animals dug into a dike carrying rails. The national railway company said the line would be out of service for at least a week.
The digging means "the rails can subside and then the safety of train traffic can no longer be guaranteed," ProRail, the company that maintains the Dutch rail network said in a statement.
Earlier this month, badgers also burrowed under tracks near the northern village of Molkwerum in Friesland province, knocking a line out of service until next month while workers seek permission to shift the animals.
Badgers are protected animals in the Netherlands, so rail operators have to get permission to move them or disturb their habitat before repairs can begin.
"""

In [12]:
instruction = "Give a summary of this text: {}"
predict_for_instruction(instruction, text, model)

'Badgers burrowing under rail tracks in the Netherlands have halted trains for at least a week.'

In [13]:
instruction = "Give a very short summary of this text: {}"
predict_for_instruction(instruction, text, model)

'Badgers burrowed under rail tracks in northern and southern Netherlands, forcing lengthy cancellations on at least two lines'

In [14]:
instruction = "Write a title for the following text:{}"
predict_for_instruction(instruction, text, model)

'Badgers halted in northern and southern Netherlands'

In [15]:
instruction = "Suggest keywords for this text. Text: {}"
predict_for_instruction(instruction, text, model)

'animal, stop, train'

## InstructGPT

FLAN модели работали хорошо, но все еще недостаточно. OpenAI довел их до состояния, когда их можно использовать на практике. Они публиковали несколько статей и описаний своих экспериментов:

https://openai.com/research/improving-language-model-behavior
https://openai.com/research/instruction-following
https://cdn.openai.com/papers/Training_language_models_to_follow_instructions_with_human_feedback.pdf
https://openai.com/research/learning-to-summarize-with-human-feedback

Они добавили еще одну важную часть - RLHF. Про нее мы попытаемся поговорить на следующем занятии. 
Пока сфокусируемся на инструкциях. Из описания OpenAI видно, что их подход очень похож на FLAN, но они машстабировали его и использовали для своих датасетов инструкции на основе реальных запросов к их API. И они продолжают это делать, исправляя все больше ошибок и нежелательных ответов. 
Также они сильно ускорились, когда добавили интерфейс (ChatGPT). Они даже говорили, что уже очень хорошая модель была доступна в их API около полугода и никто особо не обращал внимания на нее, хотя она уже работала как ChatGPT, но ей нужно было подавать правильный промпт. Когда они решили это через интерфейс (и промпт на бекенде), количество пользователей сильно увиличилось и к ним потекло очень много реальных запросов, на которых они быстро стали дообучаться.

Открытых моделей тут нет, поэтому перейдем к следующему шагу.

## Alpaca 

Разнообразние и естественность инструкция влияет на качество модели, но создавать такие датасеты сложно и дорого, а корпорации не делятся. Поэтому многие работы в обучении на инструкциях посвящены способам сгенерировать синтетические, но как можно более реалистичные датасеты. Значимая работа в этом направлении - Stanford Alpaca 
![](https://crfm.stanford.edu/static/img/posts/2023-03-13-alpaca/alpaca_main.jpg)

Код и датасет можно найти тут - https://github.com/tatsu-lab/stanford_alpaca
Дальше код взят из train.py и немного изменен

Авторы Альпаки дообучили модель LLaMA (7 миллиардов параметров) на датасете инструкций, который они сгенерировали с помощью OpenAI API и получилась модель, которая очень похожа по качеству на саму модель от OpenAI.   

LLaMa - это серия предобученных моделей от Meta. Они были опубликованы в 2023 году (LLaMA 1 в феврале, а LLaMA 2 в июле) и Meta утверждает, что по метрикам их меньшие модели сравнимы с GPT-3 (которая около 175 млрд параметров). LLaMA 2 долгое время держалась в топе открытых моделей, где конкуренцию ей составляют Mistral, DBRX, Grok-1.  


Незадолго до релиза LLaMA Meta сталкивалась с критикой за свою модель Galactica, которая была предобучена на научных статьях. Сначала они выложили её в открытый доступ, но быстро оказалась, что она может генерировать псевдонаучные и лженаучные тексты и Meta быстро закрыла доступ к этой модели. Поэтому модель LLaMA строго говоря не выложена в открытый доступ и имеет некомерческую лицензию. Чтобы скачать модель, нужно заполнять специальную форму и ждать пока ее одобрят. Но естественно люди, которые получили доступ к модели начали выкладывать ее в открытый доступ - например, кто-то делал ПР в либу Meta, в котором предлагается добавить в Readme.md ссылку на [торент](https://github.com/facebookresearch/llama/pull/73/commits/016a53608c5eae1021e171b9c4f06a9783fc14c0) 
LLaMA 2 также имеет некомерческую лицензию и требует заявки на лиценцию, однако одобрение лицензии занимает небольшое время, поэтому модель уже воспринимается как полностью открытая.

Датасет инструкций Alpaca сгенерировали на основе статьи - https://arxiv.org/abs/2212.10560 И как они говорят у них ушло около 500$ на все, что в тысячи раз дешевле того, что, предполагается, потратил сам OpenAI на свои модели. Но OpenAI запрещают использовать свои модели в таких целях и поэтому итоговую модель Alpaca они пока не выкладывают.

Но они выложили в открытый доступ датасет и можно самому попробовать дообучить какую-то открытую предобученную модель. Коммерческая применимость такой модели, однако, все еще под вопросом. 

In [16]:
# Скачаем датасет

In [18]:
# !wget https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json

In [1]:
# import copy
# import logging
# from dataclasses import dataclass, field
# from typing import Optional, Dict, Sequence
import json
import torch
import transformers
# from torch.utils.data import Dataset
from transformers import Trainer
from datasets import Dataset
# import utils
from transformers import AutoModelForCausalLM, AutoTokenizer

Посмотрим на датасет.

In [2]:
data_alpaca = json.load(open('alpaca_data.json'))

data_alpaca[:10]

[{'instruction': 'Give three tips for staying healthy.',
  'input': '',
  'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'},
 {'instruction': 'What are the three primary colors?',
  'input': '',
  'output': 'The three primary colors are red, blue, and yellow.'},
 {'instruction': 'Describe the structure of an atom.',
  'input': '',
  'output': 'An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.'},
 {'instruction': 'How can we reduce air pollution?',
  'input': '',
  'output': 'There are a number of ways to reduce air pollution, such

In [3]:
dataset = Dataset.from_list(data_alpaca)

In [4]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 52002
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [7]:
def create_completion(batch):
    outputs = batch['output']
    inputs = batch['input']
    instructions = batch['instruction']
    texts = []
    for inp, inst, out in zip(inputs, instructions, outputs):
        if inp:
            text = "\n".join([inst,inp, "Response:", out])
        else:
            text = "\n".join([inst, "Response:", out])
        texts.append(text)
    return tokenizer(texts) | {'text': texts}

dataset = dataset.map(create_completion, batched=True)

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [8]:
print(dataset['text'][0])

Give three tips for staying healthy.
Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.


In [9]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-350m", 
    cache_dir='./models'
)

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [14]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=600,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, )
)

In [15]:
trainer.train()

Step,Training Loss
1,1.882700
2,1.836600
3,2.299300
4,1.864500
5,1.649300
6,1.666400
7,1.421200
8,1.415000
9,1.311900
10,1.362500


TrainOutput(global_step=600, training_loss=2.5098100713888805, metrics={'train_runtime': 130.6271, 'train_samples_per_second': 73.492, 'train_steps_per_second': 4.593, 'total_flos': 2512808564097024.0, 'train_loss': 2.5098100713888805, 'epoch': 0.18460118452426735})

In [18]:
model.save_pretrained('opt_350_full_ft')

In [17]:
def generate(text, tokenizer, model):
    batch = tokenizer(text, return_tensors='pt').to('cuda')
    output_tokens = model.generate(**batch, max_new_tokens=100, temperature=0.1, do_sample=True, no_repeat_ngram_size=3)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [22]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-350m", 
    cache_dir='./models',
    device_map='cuda'
)

In [23]:
text = """
Badgers burrowing under rail tracks have halted trains in the northern and southern Netherlands, forcing lengthy cancellations on at least two lines.
All trains were halted Tuesday afternoon on a busy line between the southern cities of Den Bosch and Boxtel after the animals dug into a dike carrying rails. The national railway company said the line would be out of service for at least a week.
The digging means "the rails can subside and then the safety of train traffic can no longer be guaranteed," ProRail, the company that maintains the Dutch rail network said in a statement.
Earlier this month, badgers also burrowed under tracks near the northern village of Molkwerum in Friesland province, knocking a line out of service until next month while workers seek permission to shift the animals.
Badgers are protected animals in the Netherlands, so rail operators have to get permission to move them or disturb their habitat before repairs can begin.
"""

In [25]:
print(generate("Suggest how can I improve my life. \nResponse:", tokenizer, model))

Suggest how can I improve my life. 
Response:

I think you're missing the point.

The point is that you can't just "improve your life" by doing something that is not a good idea.
If you want to improve your life, you need to do something that you are passionate about.
You need to be passionate about something.
And you need a passion for something. You need to have a passion that you have to work for.
So, you can do something you are not passionate about, but


In [26]:
print(generate(f"Summarize this text. \n{text} \nResponse:", tokenizer, model))

Summarize this text. 

Badgers burrowing under rail tracks have halted trains in the northern and southern Netherlands, forcing lengthy cancellations on at least two lines.
All trains were halted Tuesday afternoon on a busy line between the southern cities of Den Bosch and Boxtel after the animals dug into a dike carrying rails. The national railway company said the line would be out of service for at least a week.
The digging means "the rails can subside and then the safety of train traffic can no longer be guaranteed," ProRail, the company that maintains the Dutch rail network said in a statement.
Earlier this month, badgers also burrowed under tracks near the northern village of Molkwerum in Friesland province, knocking a line out of service until next month while workers seek permission to shift the animals.
Badgers are protected animals in the Netherlands, so rail operators have to get permission to move them or disturb their habitat before repairs can begin.
 
Response:

The Dutc

In [27]:
model = AutoModelForCausalLM.from_pretrained(
    "opt_350_full_ft", 
    cache_dir='./models',
    device_map='cuda'
)

In [28]:
print(generate("Suggest how can I improve my life. \nResponse:", tokenizer, model))

Suggest how can I improve my life. 
Response:
One way to improve your life is to focus on your goals and make sure your goals are met. This can include setting realistic goals, taking regular breaks, and taking regular walks. Additionally, it is important to be aware of your environment and to be willing to take risks and take risks. Finally, it's important to take time to reflect on your accomplishments and to take care of yourself. Finally and most importantly, it’s important to remember that you are capable of achieving anything. Your


In [29]:
print(generate(f"Summarize this text. \n{text} \nResponse:", tokenizer, model))

Summarize this text. 

Badgers burrowing under rail tracks have halted trains in the northern and southern Netherlands, forcing lengthy cancellations on at least two lines.
All trains were halted Tuesday afternoon on a busy line between the southern cities of Den Bosch and Boxtel after the animals dug into a dike carrying rails. The national railway company said the line would be out of service for at least a week.
The digging means "the rails can subside and then the safety of train traffic can no longer be guaranteed," ProRail, the company that maintains the Dutch rail network said in a statement.
Earlier this month, badgers also burrowed under tracks near the northern village of Molkwerum in Friesland province, knocking a line out of service until next month while workers seek permission to shift the animals.
Badgers are protected animals in the Netherlands, so rail operators have to get permission to move them or disturb their habitat before repairs can begin.
 
Response:
Badger bu

## Dolly

Единственный действительно открытый и от руки написанный датасет инструкций - это databricks/databricks-dolly-15k. Этот датасет вручную написали сотрудники компании Databricks. Он выложен в открытый доступ и его разрешено использовать в коммерческих целях. 
В тот момент казалось, что таких датасетов станет гораздо больше. На основе долли/альпаки быстро файюнтинились открытые предобученные модели и они даже неплохо работали, но почему-то хороших вручную написанных датасетов больше никто не опубликовал. Возможно это просто очень дорого, или же компании поняли, что на этом можно заработать и лучше не делится с конкурентами. Новые модели часто выходят уже с instruct fine-tuned версиями, но данные никто не раскрывает (например, в релизе `databricks/dbrx-instruct` вообще нет даже описания использованного датасета!)

In [ ]:
from datasets import load_dataset

data_dolly = load_dataset("databricks/databricks-dolly-15k")

data_dolly

In [ ]:
data_dolly['train'][12]

# Parameter efficient fine-tuning

В рамках семинаров мы ограничены одной не очень большой GPU доступной на колабе (t4 с 16 гб памяти). Выше мы использовали модель с 350m параметров, иначе мы бы столкнулись с OOM ошибкой или слишком долгим обучением. Естественно качество таких моделей не впечатляет и хотелось бы попробовать модели побольше. Даже сильно побольше, так как кажется, что [эмержентные](https://ru.wikipedia.org/wiki/%D0%AD%D0%BC%D0%B5%D1%80%D0%B4%D0%B6%D0%B5%D0%BD%D1%82%D0%BD%D0%BE%D1%81%D1%82%D1%8C)
свойства, о которых все сейчас говорят, начинают проявлятся у моделей размером около 6-10 миллиардов параметров (https://arxiv.org/pdf/2206.07682.pdf). Модели поменьше тоже выкладываются, но они в любом случае начинаются от 1B. 

По умолчанию модель 7B требует около 25 гб видеопамяти, то есть даже для инференса ресурсов колаба не хватит, не говоря даже о обучении (для него понадобится в 4 раза больше).

К счастью нехватка ресурсов - общая проблема. Даже те, у кого есть такие ресурсы заинтересованы в оптимизации (можно использовать меньше ресурсов=денег или же использовать такое же количество ресурсов, но обслуживать больше пользователей=зарабатывать больше денег). Поэтому усилия многих исследователей и компаний направлены в сторону оптимизации больших языковых моделей.


В этом семинаре мы разберем несколько уже разработанных подходов и сможем обучить запустить `Qwen/Qwen2.5-7B` и дообучим `allenai/OLMo-1B-hf`!


Оптимизация генерации (inference) и оптимизация процесса дообучения это немного разные вещи, поэтому разберем их по очереди.


## Оптимизация генерации 
Для уменьшения готовой модели есть два основных подхода: дистиляция и квантизация.

**Дистиляция** (knowledge distillation) - это обучение меньшей по размеру модели воспроизводить предсказания большей модели. Исходная модель при таком подходе называется учителем, а меньшая модель - учеником. Например, можно взять большой предобученный классификатор тональности, сделать предсказания на каком-нибудь большом корпусе и обучить меньшую модель на этих предсказаниях. Так как при уменьшении количества параметров будет уменьшаться и качество, можно ограничить обучающий корпус каким-то одним доменом, чтобы упростить меньшей модели задачу (меньшая модель, например, научится хорошо определять тональность коротких текстов, но будет плохо работать с длинными; на практике это может быть приемлимый трейдофф)

Схема дистилляции:
![](https://miro.medium.com/v2/resize:fit:936/1*8KqNtABnNXM527JK9UuBUQ.jpeg)

Применять дистиляцию к простым языковым моделям нет особого смысла, так как обучающие данные и так доступны и можно просто обучить меньшую модель с нуля аналогично большой модели. Но уместна дистиляция для моделей, обученных на инструкциях. Вот например эксперименты по дистиляции GPT в небольшие модели - https://github.com/mbzuai-nlp/LaMini-LM В целом это очень похоже на [Alpaca](https://github.com/tatsu-lab/stanford_alpaca), но датасет здесь намного больше, а сама модель обучается с нуля (в Alpaca дообучается Llama). И также как и с Alpaca, это не очень то легально - по сути это попытка скопировать модель без доступа к обучающим данным, поэтому лицензия на моделях запрещает коммерческое использование.


**Квантизация** - это уменьшение размера модели за счет уменьшения точности представления чисел. Веса в модели это просто очень много чисел вида 0.23123125, -1.234559 и для хранения каждого такого числа требуется какое-то количество памяти. Интервал допустимых значений и количество знаков после запятой всегда ограничены, но по умолчанию они достаточно большие и, выясняется, что можно достаточно сильно округлить веса, и при этом, практически не потерять в качестве!
Схема квантизации:
![](https://developer-blogs.nvidia.com/wp-content/uploads/2021/07/qat-training-precision.png)

Разумеется, квантизация сложнее, чем просто округление, но подробно методы квантизации мы разбирать не будем. Если вам интересно, можно почитать вот это - https://huggingface.co/blog/hf-bitsandbytes-integration Один из авторов - Tim Dettmers, автор библиотеки bitsandbytes, в которой реализованы методы квантизации и которая постепенно интегрируется в huggingface.

Давайте посмотрим как использовать квантизацию в huggingface с предобученными моделями.

In [4]:
# !pip install -q git+https://github.com/huggingface/transformers.git@main
# !pip install accelerate

In [3]:
# !pip install torch torchvision torchaudio -U

In [11]:
# !nvidia-smi

In [5]:
import torch; print(torch.__version__)

/opt/conda/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


2.6.0+cu124


Попробуем загрузить большую модель без дополнительных параметров.

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [7]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='Qwen/Qwen2.5-3B',
                                             cache_dir='./models').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B")

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [8]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In the beginning the Universe was created. It was very hot and very dense. Then the Big Bang happened. The Universe expanded and cooled. After a few minutes, the first atoms formed. They were mostly hydrogen and helium. By the time stars formed, about 100 million years


Можно посмотреть сколько такая модель занимает памяти на gpu. Модель с 3B параметров занимает около 12гб видеопамяти

In [9]:
!nvidia-smi

Tue Mar 18 15:55:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:01:00.0 Off |                  N/A |
|  0%   53C    P8             22W /  285W |   12215MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


По умолчанию веса хранятся в fp32 формате. Модель `Qwen2.5-7B` при таком формате не поместится в память.

In [10]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='Qwen/Qwen2.5-7B',
                                             cache_dir='./models').to('cuda')

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 260.00 MiB. GPU 0 has a total capacity of 15.58 GiB of which 123.00 MiB is free. Process 691591 has 15.30 GiB memory in use. Of the allocated memory 15.03 GiB is allocated by PyTorch, and 26.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### Не забудьте перезагрузить кернел после OOM ошибки!

## FP16

Давайте попробуем загрузить модель в fp16 (из названия можно догадаться, что этот формат в два раза меньше)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

/opt/conda/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='Qwen/Qwen2.5-3B',
                                             torch_dtype=torch.float16, # указываем fp16
                                             cache_dir='./models').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In the beginning the Universe was created. It was very hot and dense. Then it expanded and cooled. As it cooled, it became less dense and different kinds of particles formed. The Universe became a hot soup of quarks and gluons. This soup was called quark-gluon


Теперь занято около 6гб!

In [5]:
!nvidia-smi

Tue Mar 18 16:00:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:01:00.0 Off |                  N/A |
| 34%   57C    P8             45W /  285W |    6585MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Давайте попробуем модель с 7B параметрами еще раз

In [2]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='Qwen/Qwen2.5-7B',
                                             torch_dtype=torch.float16, # указываем fp16
                                             cache_dir='./models').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [3]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.
-- Douglas Adams, The Hitchhiker's Guide to the Galaxy
The Big Bang theory is the most widely accepted scientific explanation for the origin of the universe.


#### Работает! Если посмотреть на память, то увидим 15ГБ (что очень близко к пределу)

In [4]:
!nvidia-smi

Tue Mar 18 16:03:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:01:00.0 Off |                  N/A |
| 33%   57C    P0             50W /  285W |   15125MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## 8-bit

Можно пойти еще дальше и попробовать 8-битный формат, который требует в ~3 раза меньше памяти. Так мы можем попробовать даже еще большую модель

In [2]:
# такие квантизации работают через дополнительную библиотеку bitsandbytes
# !pip install bitsandbytes

In [3]:
# !pip install --upgrade 'optree>=0.13.0'

In [9]:
# !apt install -y cmake

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [2]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

In [3]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='Qwen/Qwen2.5-7B',
                                             quantization_config=quantization_config,
                                             device_map='auto',
                                             cache_dir='./models')
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.
-- Douglas Adams, The Hitchhiker's Guide to the Galaxy
The Big Bang theory is the most widely accepted scientific explanation for the origin of the universe.


In [5]:
# память снизилась даже немного больше чем в два раза
!nvidia-smi

Tue Mar 18 16:06:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:01:00.0 Off |                  N/A |
| 33%   54C    P0             50W /  285W |    9053MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
+---------------------------------------------------------------------------------

## 4 bit
Можно пойти и еще дальше и сделать 4bit квантизацию! Но тут уже все становится сложнее и память снижается не так сильно. Веса из floating point формата нельзя просто нормализовать до целых чисел и работать с ними также как и раньше. 8bit и 4bit квантизации нормализуют веса в целые числа, чтобы они занимали меньше памяти, но сами вычисления производятся в floating point форматах - для этого перед вычислением нужная матрица деквантизируется в fp16/fp32 формат, а затем стирается из памяти, чтобы освободить место для следующей матрицы. Естественно такой подход медленее. И в целом квантизация экономит память, но не ускоряет генерацию! Современные видеокарты могут иметь специальные оптимизации под разные типы квантизации и скорость может сильно варьироваться (а старые видеокарты могут вообще не поддерживать квантизацию). 
Формат вычислений можно регулировать отедльным параметром.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [2]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        # torch.float16 сократит потребление памяти, но даже так модель помещается потому что веса 4битные
        bnb_4bit_compute_dtype=torch.float32, 
        # есть еще дополнительные advanced параметры которые могут снижать потребление памяти
        # про них можно почитать тут - https://github.com/huggingface/blog/blob/main/4bit-transformers-bitsandbytes.md
        bnb_4bit_use_double_quant=True
    )

In [3]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='Qwen/Qwen2.5-7B',
                                             quantization_config=quantization_config,
                                             device_map='auto',
                                             cache_dir='./models')
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In the beginning the Universe was created. The Universe is everything there is. It has always existed. Everything in the universe is made of matter. Matter can be anything you can touch or see. Energy is also part of the matter in our universe. Light is energy and matter too. There


In [5]:
!nvidia-smi

Tue Mar 18 16:07:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


|   0  NVIDIA GeForce RTX 4070 ...    On  |   00000000:01:00.0 Off |                  N/A |
| 33%   57C    P0            149W /  285W |    6179MiB /  16376MiB |     79%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
+---------------------------------------------------------------------------------

## Оптимизация обучения

Квантизация также применима при обучении моделей, но ограничено. Как правило обучают в mixed формате, когда используется float16 и float32. 8bit и 4bit пока не подходят для обучения. 


Согласно вычислениями вот отсюда - https://blog.eleuther.ai/transformer-math/ - обучение требует в ~4 раза больше памяти, чем inferece и даже если мы можем загрузить 7B в float16 не значит, что мы сможем ее обучать. 
Даже с квантизированной моделью полный файн-тюнинг - это все еще очень дорого. 

Но у нас есть еще одна опция, которая позволяет пользоваться и 8bit и 4bit квантизацией при обучении! Эта опция - адаптеры. 
Самый популярный на данный момент - LoRA ( https://arxiv.org/abs/2106.09685 ). Мы уже использовали частичное дообучение на предыдущих занятиях, когда добавляли 1 дополнительный полносвязный слой к предобученной модели и обучали только его. LoRA - это обобщение такого подхода, но тут, дополнительные веса можно добавить к каждому полносвязному слою в предобученной модели. 

На одном из прошлых семинаров мы разбирали, что большую матрицу можно представить как произведение нескольких матриц поменьше. Причем внутреннюю размерность этих матриц можно варьировать - чем меньше, тем хуже восстанавливается оригинальная матрица, но тем меньше памяти занимается. В LoRA (low rank adapter) используется такой же прием - к каждой матрице с весами (полносвязному слою) прибавляется матрица такого же размера, которая получена произведением двух матриц поменьше A и B. Внутренюю размерность этих матриц можно варьировать - это гиперпараметр модели (см. ниже), чем она больше, тем выше точность модели, и тем выше потребность в памяти. Такой трюк к тому же обоснован - авторы LoRA проанализировали веса в предобученных нейронных сетях и заметили, что большинство из них имеют низкий ранг! То есть дополнительный адаптер может добавить в модель сопоставимое количество информации.

Схема LoRA:
![](https://miro.medium.com/v2/resize:fit:730/1*D_i25E9dTd_5HMa45zITSg.png)


При обучении веса основной модели замораживаются. При небольшой внутренней размерности получается, что дообучение адаптера требует лишь небольшой процент ресурсов, требуемых для полноценного обучения. И так как для основной модели не требуется расчитывать градиенты и обновлять веса, ее можно загружать в 8/4bit формате!

Также полученные веса очень удобно использовать на практике - их можно сохранять и подгружать отдельно, не затрагивая оригинальною большую модель. Предобученные модели весят гигабайты, а LoRA веса - несколько мегабайт. Можно также дообучить несколько LoRA весов и переключаться между ними на лету.

LoRA реализована в отдельной библиотеке PEFT в экосистеме huggingface. Давайте попробуем дообучить большую модель

### Fine-tuning

In [6]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [2]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

В итоге я использую модель `allenai/OLMo-1B-hf` модель как пример, но 7B (и потенциально большие) модели тоже работают, просто обучаются дольше

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "allenai/OLMo-1B-hf", 
    quantization_config=quantization_config,
    cache_dir='./models'
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [13]:
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-hf")

tokenizer_config.json:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

В этой ячейке все веса изначальной модели замораживаются

In [14]:
for param in model.parameters():
  param.requires_grad = False  
  if param.ndim == 1:
    # в layernorm нужны очень маленькие числа, поэтому для него оставляют fp32 
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [15]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

Адаптеры можно добавлять к полносвязным/линейным/dense слоям. В зависимости от архитектуры эти слои могут называться по-разному, поэтому их нужно указать вручную через парметр `target_modules`. Так как модели это в основном наслоенные однотипные трансформерные блоки, то перечислить нужно лишь несколько имен типовых слоев. 

In [16]:
# вот так можно (в torch) напечатать слои и их названия
# в нашем случае на них прямо написано, что они линейные (proj)
# поэтому мы можем выбрать q_proj, v_proj, k_proj, o_proj, gate_proj, up_proj, down_proj
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp
model.layers.2.mlp.gate_proj
model.l

In [17]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32, # внутренняя размерность адаптера, основной параметр
    target_modules=["q_proj", "k_proj", "v_proj", 'o_proj', 'gate_proj',  'up_proj', 'down_proj'], # к каким слоям добавлять адаптеры (подробнее выше)

    # "вес" адаптера, этот параметр делится на r, то есть если они равны то
    # вес адаптера = 1 (то есть базовая модель и адаптер одинаковы по значимости)
    # если поставить этот параметр выше, то адаптер будет сильнее влиять на базовую модель
    # как я понимаю никто особо не понимает что делать с этим параметром при обучении
    # лучше оставлять его равным r
    lora_alpha=32, 
    
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 24117248 || all params: 1200881664 || trainable%: 2.0082951320672344


Параметр r отвечает за внутренюю размерность дополнительных матриц. При r=32 обучаться в итоге будет около 12м параметров, что около 2 % от всех параметров изначальной модели.

### Обучение

In [18]:
import transformers
from datasets import load_dataset

def create_completion(batch):
    outputs = batch['output']
    inputs = batch['input']
    instructions = batch['instruction']
    texts = []
    for inp, inst, out in zip(inputs, instructions, outputs):
        text = "\n".join([inst, "Response:", inp, out])
        texts.append(text)
    return tokenizer(texts) | {'text': texts}

# в качестве датасета я взял инструкции к генерации кода
data = load_dataset("flytech/python-codes-25k", split='train[:20%]')
# data = data.map(lambda samples: tokenizer(samples['text']), batched=True)
data = data.map(create_completion, batched=True)

Map:   0%|          | 0/9925 [00:00<?, ? examples/s]

Форматированный пример для обучения выглядит так. Я добавил `Response:` как разделитель запроса и ответа, чтобы потом можно было подавать его как часть промпта и ожидать, что модель сразу начнет генерировать ответ. (в реальных моделях для этого вводят специальные токены)

In [19]:
print(data[0]['text'])

Help me set up my daily to-do list!
Response:
Setting up your daily to-do list...
```python
tasks = []
while True:
    task = input('Enter a task or type 'done' to finish: ')
    if task == 'done': break
    tasks.append(task)
print(f'Your to-do list for today: {tasks}')
```


In [20]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=600,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, )
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [21]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
1,1.384500
2,1.501600
3,1.625300
4,1.699800
5,1.205700
6,1.443900
7,1.309600
8,1.638400
9,1.503500
10,1.643900


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

TrainOutput(global_step=600, training_loss=0.8256732753912608, metrics={'train_runtime': 395.7639, 'train_samples_per_second': 24.257, 'train_steps_per_second': 1.516, 'total_flos': 1.2278139677835264e+16, 'train_loss': 0.8256732753912608, 'epoch': 0.9669621273166801})

Сохраним ~~модель~~ адаптер (сохранятся только дополнительные веса)

In [22]:
model.save_pretrained('olmo_lora')

Чтобы загрузить обученный адаптер нужно сначала загрузить базовую модель, а потом применить к ней LoRa веса

In [23]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

peft_model_id = "olmo_lora"

quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="allenai/OLMo-1B-hf",
                                             return_dict=True, 
                                             quantization_config=quantization_config,
                                             device_map='auto',
                                             cache_dir='./models'
                                            )
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-hf")

In [24]:
def generate(text, tokenizer, model):
    batch = tokenizer(text, return_tensors='pt').to('cuda')
    output_tokens = model.generate(**batch, max_new_tokens=100, temperature=0.1, do_sample=True, no_repeat_ngram_size=3)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

Давайте попробуем сделать несколько предсказаний, используя базовую модель, чтобы потом сравнить с дообученной

In [25]:
print(generate("Write a function to see the current day of the week. \nResponse:",  tokenizer, model))

Write a function to see the current day of the week. 
Response: The function returns the current date and time.

## Exercise 2:

Write a program to find the sum of the numbers from 1 to 100.
The program should print the sum.
You can use the following code to find out the sum:
```
sum = 0
for i in range(1,100):
    sum = sum + i
print(sum)
```


## Explanation:
The sum of numbers from one to 100 is 100


In [26]:
print(generate("Help me with a todo list! \nResponse:",  tokenizer, model))

Help me with a todo list! 
Response: I'm sorry, I don't have a todo-list.
Response 2: I don’t have a list either.
I’m sorry, but I don´t have any todo list.
You can use the following sentence to say that you don’ t have a to-do list.The first thing you need to do is to make sure that you have a good internet connection. If you have an internet connection that is not fast enough, then you will not be


Загрузим LoRA веса и попробуем те же промпты

In [27]:
model = PeftModel.from_pretrained(model, peft_model_id)

In [28]:
print(generate("Write a function to see the current day of the week. \nResponse:",  tokenizer, model))

Write a function to see the current day of the week. 
Response:

```python
import datetime

def get_current_weekday():
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return weekdays[datetime.date.today().weekday()]
```

# Output: 'Monday'
``` python
import dateutil.parser

current_date = datetime.date(2020, 1, 1)
weekday = get_


In [29]:
print(generate("Help me with a todo list! \nResponse:",  tokenizer, model))

Help me with a todo list! 
Response:

```python
import tkinter as tk
root = tk.Tk()
root.title('Todo List')
# Create a form for adding tasks
form = ttk.Form(root)
# Set the label for the form
label = tttk.Label(form, text='Add a task here')
label.pack()
# Add a button to add a task
button = ttks.Button(form)



Теперь модель генерирует код по инструкции! 